In [ ]:
LIST_OF_DIRECTORIES = ["output/haut1c/coffee", "output/bas1c/coffee", "output/milieu1c/coffee", "output/droite1c/coffee", 
                      "output/gauche1c/coffee", "output/bas1c/ambiguous/coffee", "output/haut1c/ambiguous/coffee", 
                      "output/gauche1c/ambiguous/coffee", "output/gauche1c/pepper", "output/gauche1c/nativevege", 
                      "output/bas1c/native_no_tree", "output/haut1c/native_no_tree", "output/gauche1c/native_no_tree",
                      "output/droite1c/native_no_tree", "output/milieu1c/native_no_tree", "output/haut1c/urban",
                      "output/bas1c/rice", "output/bas1c/tea", "output/bas1c/water", "output/haut1c/water",
                      "output/milieu1c/water", "output/gauche1c/water", "output/droite1c/water", 
                      "output/gauche1c/rubber", "output/bas1c/pine_trees", "output/haut1c/deciduous_forest", 
                      "output/gauche1c/intercrop", "output/droite1c/seasonal", "output/haut1c/nativevege", 
                      "output/droite1c/other_tree", "output/milieu1c/other_tree", "output/haut1c/cassava", 
                      "output/milieu1c/cassava"]
COUNTAIN_START = "coffee"
PIXEL_LENGTH = 15
BAND_WIDTH = 1
NB_SPLIT = 5
EPOCH = 500

# Première étape : charger les données

Importer le nom des données

In [ ]:
import os 
import numpy as np
from matplotlib import pyplot as pl
import random
import rasterio
os.environ['CUDA_VISIBLE_DEVICES']='1'

%matplotlib inline

In [ ]:
def get_all_images_names(directories): 
    images_name = []
    for directory in directories:
        counter = 0
        for filename in os.listdir(directory): 
            if os.path.isfile(directory + "/" + filename):
                images_name.append(directory + "/" + filename)
                counter += 1
            elif os.path.isdir(directory + "/" + filename):
                images_name += get_all_images_names([directory + "/" + filename])
        print(directory, counter, "images.")
    return images_name
images_name = get_all_images_names(LIST_OF_DIRECTORIES)

Importer les images et leur résultat voulu

In [ ]:
def reshape(image):
    """Reformate les données rasterio pour être utilisée par Tensorflow"""
    reshaped_image = []
    for i in range(len(image[0])):
        reshaped_row = []
        for j in range(len(image[0][0])):
            reshaped_cell = []
            #Automne
            reshaped_cell.append(image[0][i][j] / 65535) #Infra rouge
            reshaped_cell.append(image[1][i][j] / 65535) #Rouge
            reshaped_cell.append(image[2][i][j] / 65535) #Vert
            reshaped_cell.append(image[3][i][j] / 65535) #Bleu
            #Hiver
            reshaped_cell.append(image[4][i][j] / 65535) #Infra rouge
            reshaped_cell.append(image[5][i][j] / 65535) #Rouge
            reshaped_cell.append(image[6][i][j] / 65535) #Vert
            reshaped_cell.append(image[7][i][j] / 65535) #Bleu
            #Printemps
            reshaped_cell.append(image[8][i][j] / 65535) #Infra rouge
            reshaped_cell.append(image[9][i][j] / 65535) #Rouge
            reshaped_cell.append(image[10][i][j] / 65535) #Vert
            reshaped_cell.append(image[11][i][j] / 65535) #Bleu
            
            #Si on utilise des images n'utilisant pas toutes les bandes (la séléction des bandes a déjà étée faites)
            #utilisez plutôt le code ci-dessous
            #for k in range(BAND_WIDTH):
            #    reshaped_cell.append(image[k][i][j] / 65535)
            
            reshaped_row.append(reshaped_cell)
        reshaped_image.append(reshaped_row)
    return reshaped_image

In [ ]:
images = []
for image_name in images_name:
    if COUNTAIN_START in image_name:
        images.append((np.array(rasterio.open(image_name).read()), 1))
    else:
        images.append((np.array(rasterio.open(image_name).read()), 0))
print("Longueur initiale : ", len(images))

In [ ]:
def add_all_rotations(images):
    result = []
    for image in images:
        result.append(image)
        for i in range (1, 4):
            rotated_image = []
            #Si on utilise des images n'utilisant pas toutes les bandes (la séléction des bandes a déjà étée faites)
            #utilisez BAND_WIDTH à la place de 12
            for j in range(12): 
                rotated_image.append(np.rot90(image[0][j], i))
            result.append((rotated_image, image[1]))
    return result

In [ ]:
images = add_all_rotations(images)
random.shuffle(images)

In [ ]:
images_x = []
images_y = []
for image in images:
    images_x.append(reshape(image[0]))
    images_y.append(image[1])
images_x = np.array(images_x).reshape(len(images_x), PIXEL_LENGTH, PIXEL_LENGTH, BAND_WIDTH)
images_y = np.array(images_y)
print("Longueur du set d'entrainement : ", len(images_x))

# Deuxième étape : créer notre cross validation et les poids de nos classes

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics as me

In [ ]:
kfold = StratifiedKFold(n_splits=NB_SPLIT)

In [ ]:
class_weight = None
pas_cafe = np.count_nonzero(images_y < 1)
cafe = len(images_y) - pas_cafe
print(pas_cafe, len(images_y))
print(pas_cafe / len(images_y))
if pas_cafe < cafe:
    class_weight = {0: cafe / pas_cafe,1: 1.} 
else:
    class_weight = {0: 1., 1: pas_cafe / cafe} 

# Troisième étape : entrainer et tester

In [ ]:
def get_fscore_recall_precision(truth, prediction):
    true_positive = 0
    false_negative = 0
    false_positive = 0
    for i in range(len(truth)):
        if truth[i] == 1 and prediction[i] == 1:
            true_positive += 1
        elif truth[i] == 0 and prediction[i] == 1:
            false_positive += 1
        elif truth[i] == 1 and prediction[i] == 0:
            false_negative += 1
    precision = true_positive/(true_positive + false_positive) if (true_positive + false_positive) > 0 else None
    recall = true_positive/(true_positive + false_negative) if (true_positive + false_negative) > 0 else None
    fscore = 2 * recall * precision / (recall + precision) if recall != None and precision != None else None
    return fscore, recall, precision

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
avg_scores = []
avg_recall = []
avg_precision = []
avg_fscore = []
i = 0
for train, test in kfold.split(images_x, images_y):
    # create model
    model = Sequential()
    model.add(Conv2D(32, (3,3), activation='relu',  input_shape=(PIXEL_LENGTH,PIXEL_LENGTH,BAND_WIDTH)))
    model.add(MaxPooling2D(pool_size = (2 ,2)))
    model.add(Conv2D(16, (3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size = (2 ,2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # Fit the model
    history = model.fit(images_x[train], images_y[train], epochs=EPOCH, batch_size=32, 
                        validation_data=(images_x[test], images_y[test]), class_weight=class_weight)
    # evaluate the model
    scores = model.evaluate(images_x[test], images_y[test], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    avg_scores.append(scores[1] * 100)
    model.save("model"+str(i)+".h5")
    print("Iteration : ", i)
    i += 1
    
    pl.plot(history.history['loss'], label='Training')
    pl.plot(history.history['val_loss'], label='Testing')
    pl.ylim(0, 2) 
    pl.legend()
    pl.grid()
    print('Test score:', scores[0])
    print('Test accuracy:', scores[1])
    pl.show()
    
    pred = model.predict_classes(images_x[test])
    f, r, p = get_fscore_recall_precision(images_y[test], pred)
    print('Test recall', r)
    print('Test precision', p)
    print('Test f-score', f)
    avg_recall.append(r * 100)
    avg_precision.append(p * 100)
    avg_fscore.append(f * 100)
    print(me.confusion_matrix(images_y[test], pred))
    
print("Accuracy %.2f%% (+/- %.2f%%)" % (np.mean(avg_scores), np.std(avg_scores)))
print("Recall %.2f%% (+/- %.2f%%)" % (np.mean(avg_recall), np.std(avg_recall)))
print("Precision %.2f%% (+/- %.2f%%)" % (np.mean(avg_precision), np.std(avg_precision)))
print("F-Score %.2f%% (+/- %.2f%%)" % (np.mean(avg_fscore), np.std(avg_fscore)))